In [2]:
# 1.1 Importing libraries
import pandas as pd
import numpy as np
import os

In [3]:
# 1.2  Importing tables
path = r'C:\Users\Lenovo\Instacart Basket Analysis'

In [ ]:
df_ords_prods_merge = pd.read_pickle(os.path.join (path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [ ]:
df = df_ords_prods_merge[:1000000]

In [5]:
# 1.3 checking

In [6]:
df.head()

,Unnamed: 0_x,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,_merge
0,0,2539329,1,prior,1,2,8,NaN,196,1,0,195,Soda,77,7,9.0,both
1,1,2398795,1,prior,2,3,7,15.0,196,1,1,195,Soda,77,7,9.0,both
2,2,473747,1,prior,3,3,12,21.0,196,1,1,195,Soda,77,7,9.0,both
3,3,2254736,1,prior,4,4,7,29.0,196,1,1,195,Soda,77,7,9.0,both
4,4,431534,1,prior,5,4,15,28.0,196,1,1,195,Soda,77,7,9.0,both


In [7]:
# 1.4 Dropping unnecessary columns of Unnamed: 0_x and Unnamed: 0_y

In [ ]:
df = df.drop(columns = ['Unnamed: 0_x'])

In [ ]:
df = df.drop(columns = ['Unnamed: 0_y'])

In [10]:
df.head()

,order_id,user_id,eval_set,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both


In [11]:
df.shape

(1000000, 15)

In [12]:
# 1.5 Grouping

In [ ]:
df.groupby('product_name')

In [14]:
# 1.6 Aggregating

In [15]:
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [16]:
df.groupby('department_id')['order_number'].mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64

In [1]:
df.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

NameError: name 'df' is not defined

In [30]:
# 1.7 labeling customers by loyalty

def customer_label(row):

  if row['order_number'] <= 10:
    return 'New customer'
  elif (row['order_number'] > 10) and (row['order_number'] <= 40):
    return 'Regular customer'
  elif row['order_number'] > 40:
    return 'Loyal customer'
  else: return 'Not enough data'

df['order_number'] = df.apply(customer_label, axis=1)

df['order_number'].value_counts(dropna = False)

In [18]:
# 1.8 Aggregating with transform()

In [19]:
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
df.head(100)

In [23]:
pd.options.display.max_rows = None

In [27]:
# 1.9 labeling with loc()

In [26]:
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [28]:
df['loyalty_flag'].value_counts(dropna = False)

Regular customer    463844
Loyal customer      335086
New customer        201070
Name: loyalty_flag, dtype: int64

In [32]:
df[['user_id', 'loyalty_flag', 'order_number']].head(25)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10
